In [17]:
## We will find out 30-day 90-day and 180-day return for all price levels ($1 - $ 10) based on institutional holdings.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [18]:
daily_stock_data_filename = './../data/daily_data_2005-2021_filtered_nyse.pkl'
daily_stock_data = pd.read_pickle(daily_stock_data_filename)

daily_stock_data['trfd'].fillna(1, inplace=True)
daily_stock_data['ajexdi'].fillna(1, inplace=True)
daily_stock_data['date'] = pd.to_datetime(daily_stock_data['datadate'], format='%Y/%m/%d')
daily_stock_data['cusip_brief'] = daily_stock_data['cusip'].apply(lambda x: str(x)[:8])

In [ ]:
# To pull institutional holdings data let's get the cusip of interest for the highest price level ($10)
min_prices = daily_stock_data.groupby('cusip')['prccd'].min()
min_prices = min_prices[min_prices < 10]
stocks_to_keep = min_prices.index

# Save the stocks to keep in a text file
with open('./../data/stocks_to_keep_at_10.txt', 'w') as f:
    for stock in stocks_to_keep:
        str_to_write = str(stock)[:8] + "\n"
        f.write(str_to_write)

In [ ]:

for threshold in [5]:
    print(f"Threshold: {threshold}")

    # Keep stocks have hit threshold at some point
    min_prices = daily_stock_data.groupby('cusip')['prccd'].min()
    min_prices = min_prices[min_prices < threshold]
    stocks_to_keep = min_prices.index
    daily_stock_data_at_thresh = daily_stock_data[daily_stock_data['cusip'].isin(stocks_to_keep)]

    # Group by cusip and sort dates
    daily_stock_data_at_thresh = daily_stock_data_at_thresh.sort_values(['cusip', 'date'])

    # Get institutional holdings data
    institutional_holdings_filename = './../data/institutional_data_5_thresh.csv'
    institutional_holdings = pd.read_csv(institutional_holdings_filename)

    # groupby cusip and fdate and type and get the sum of shares
    institutional_holdings_agg = institutional_holdings.groupby(['cusip', 'fdate', 'type'])['shares'].sum().reset_index()
    # Just type == 5
    institutional_holdings_agg = institutional_holdings_agg[institutional_holdings_agg['type'] == 5]
    # Make fdate a datetime
    institutional_holdings_agg['fdate'] = pd.to_datetime(institutional_holdings_agg['fdate'], format='%Y/%m/%d')
    # make institution holdings agg daily instead of quarterly by resampling
    institutional_holdings_agg_daily = institutional_holdings_agg.set_index('fdate').groupby('cusip')['shares'].resample('D').ffill().reset_index()

    # Merge with daily stock data
    daily_stock_data_merged = daily_stock_data_at_thresh.merge(institutional_holdings_agg_daily, how='left', left_on=['cusip_brief', 'date'], right_on=['cusip', 'fdate'])

    # drop na cusip_y
    daily_stock_data_merged = daily_stock_data_merged.dropna(subset=['cusip_y'])

    # ownership percentage. shares / cshoc
    daily_stock_data_merged['instituional_ownership_perc'] = daily_stock_data_merged['shares'] / daily_stock_data_merged['cshoc']

    daily_stock_data_merged = pd.read_pickle('./../data/monthly_stock_data_simulated.pkl')

    ## RUN STRATEGY ------------------------------------------------------------------------------------------------------------------##
    i = 0
    plot = False
    plot = False

    last_date = daily_stock_data_merged['date'].max()
    allTrades = []
    tradeStartDate = np.NaN
    cusip_of_interest = ['13462K109', '92854Q106']
    # Iterate over every stock by cusip
    for index, group in daily_stock_data_merged.groupby('cusip_x'):
        i += 1

        # If state is 0, then look for a buy signal (price <= threshold)
        #   If found, set state to 1 and set tradeStartDate to the date of the buy signal, , record the ownership percentage.
        # If state is 1, then look for whether it's been 30 days, 90 days, or 180 days since the tradeStartDate
        #   If it's been 30 days, then record cumulative return.
        #   If it's been 90 days, then record cumulative return.
        #   If it's been 180 days, then record cumulative return and set state to 0.

        returns = []

        returns_agg = {
            '30': -1,
            '90': -1,
            '180': -1
        }
        trades = []

        ownershipPerc = 0
        state = 6
        prevAdjPrice = np.NaN
        for index, day in group.iterrows():
            if(state == 6 and day['prccd'] > threshold):
                state = 0
            elif(state == 0 and day['prccd'] <= threshold):
                state = 1
                tradeStartDate = day['date']
                prevAdjPrice = day['prccd'] / day['ajexdi'] * day['trfd']
                ownershipPerc = day['instituional_ownership_perc']
                returns = []

            elif(state != 0 and state != 6):
                ret = 1 + (day['prccd'] / day['ajexdi'] * day['trfd'] - prevAdjPrice) / prevAdjPrice
                prevAdjPrice = day['prccd'] / day['ajexdi'] * day['trfd']
                returns.append(ret)
                if(day['date'] - tradeStartDate >= pd.Timedelta('30 days') and state == 1):
                    returns_agg['30'] = np.prod(returns) - 1
                    print("Length of 30-day returns: {}".format(len(returns)))
                    state = 2

                elif(day['date'] - tradeStartDate >= pd.Timedelta('90 days') and state == 2):
                    returns_agg['90'] = np.prod(returns) - 1
                    print("Length of 90-day returns: {}".format(len(returns)))
                    state = 3

                elif(day['date'] - tradeStartDate >= pd.Timedelta('180 days') and state == 3):
                    returns_agg['180'] = np.prod(returns) - 1
                    print("Length of 180-day returns: {}".format(len(returns)))
                    state = 6

                    trades.append({
                        'cusip' : group['cusip_x'].iloc[0],
                        'tic' : group['tic'].iloc[0],
                        'ownershipPerc' : ownershipPerc,
                        'startDate' : tradeStartDate,
                        'returns' : returns_agg.copy()
                    })

                    returns_agg = {
                        '30' : -1,
                        '90' : -1,
                        '180' : -1
                    }
                elif(day['date'] - datetime.timedelta(days = 4) >= last_date):
                    state = 5
                    break

        # Account for delistings
        if(state != 0 and state != 5 and state != 6):
            trades.append({
                'cusip' : group['cusip_x'].iloc[0],
                'tic' : group['tic'].iloc[0],
                'ownershipPerc' : ownershipPerc,
                'startDate' : tradeStartDate,
                'returns' : returns_agg.copy()
            })

        if(plot or group['cusip_x'].iloc[0] in cusip_of_interest):
            print("For stock " + group['tic'].iloc[0] + " with ownership percentage " + str(ownershipPerc) + ", the returns are " + str(trades) + ".")
            # Plot the price
            fig, ax = plt.subplots(figsize=(20, 10))
            ax.plot(group['date'], group['prccd'])

            group = group.copy()
            group['ajexdi_change'] = group['ajexdi'] - group['ajexdi'].shift(1)
            splits = group[group['ajexdi_change'] != 0]

            if(len(splits) > 0):
                # Big markers
                colors = ['red' if x > 0 else 'green' for x in splits.iloc[1:]['ajexdi_change']]
                ax.scatter(splits.iloc[1:]['date'], splits.iloc[1:]['prccd'], marker='x', s=100, c=colors)
            # Draw horizontal dashed line at threshold
            ax.axhline(y=threshold, linestyle='--', color='green')

            # Plot institutional ownership with twin y
            # height of ax2 is 0.2 of ax
            divider = make_axes_locatable(ax)
            ax2 = divider.append_axes("bottom", size="30%", pad=0.3)

            # set ax2 height

            ax2.plot(group['date'], group['instituional_ownership_perc'], color='red')
            ax2.set_ylim(0, 1.1)
            ax2.set_ylabel('Institutional Ownership')
            
            # Add ticker to title
            ax.set_title(group.iloc[0]['tic'])
            # Add y axis label
            ax.set_ylabel('Price')

            for trade in trades:
                ax.axvspan(trade['startDate'] + pd.Timedelta('90 days'), trade['startDate'] + pd.Timedelta('180 days'), alpha=0.1, color='green')
                ax.axvspan(trade['startDate'] + pd.Timedelta('30 days'), trade['startDate'] + pd.Timedelta('90 days'), alpha=0.1, color='yellow')
                ax.axvspan(trade['startDate'] , trade['startDate'] + pd.Timedelta('30 days'), alpha=0.1, color='orange')
                # pre trade period
                ax.axvspan(trade['startDate'] - pd.Timedelta('30 days'), trade['startDate'], alpha=0.1, color='blue')

            # add legend to ax
            ax.legend(['Price', '', 'Buy threshold', '180 day period', '90 day period', '30 day period', 'Before trade start'])

            print('Cusip: ' + str(group['cusip_x'].iloc[0]))

            # plot axjedi and trfd
            fig, ax = plt.subplots(figsize=(20, 10))
            ax.plot(group['date'], group['ajexdi'], label='ajexdi')
            ax.plot(group['date'], group['trfd'], label='trfd')

            ax.legend()

            plt.show()
        

        allTrades += trades

        if(i > 10):
            break



    # Make a df out of allTrades with columns startDate, ownershipPerc, 30, 90, 180, cusip, tic
    allTradesDf = pd.DataFrame(allTrades)
    # Unwrap the returns column with dict keys as columns and dict values as values
    allTradesDf = pd.concat([allTradesDf.drop(['returns'], axis=1), allTradesDf['returns'].apply(pd.Series)], axis=1)

    # Bin trades
    allTradesDf['ownershipPercBin'] = pd.cut(allTradesDf['ownershipPerc'], [0, 0.2, 0.4, 0.6, 0.8, 1], labels=['0-20%', '20-40%', '40-60%', '60-80%', '80+'])
    # Drop nan values
    allTradesDfDroppedNan = allTradesDf
    allTradesDfDroppedNan = allTradesDfDroppedNan[allTradesDfDroppedNan['30'] < 10]

    # Print the mean and median for each bin for the returns as a table
    display(allTradesDfDroppedNan.groupby('ownershipPercBin')['30'].agg(['mean', 'median']))
    display(allTradesDfDroppedNan.groupby('ownershipPercBin')['90'].agg(['mean', 'median']))
    display(allTradesDfDroppedNan.groupby('ownershipPercBin')['180'].agg(['mean', 'median']))


    # Sort 180 day returns
    allTradesDfDroppedNan = allTradesDfDroppedNan.sort_values(by=['180'], ascending=False)

    display(allTradesDfDroppedNan)

    





    

Threshold: 5


C:\Users\amrit\AppData\Local\Temp\ipykernel_6336\590520853.py:15: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  institutional_holdings = pd.read_csv(institutional_holdings_filename)


Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180
Length of 30-day returns: 30
Length of 90-day returns: 90
Length of 180-day returns: 180


,mean,median
ownershipPercBin,,
0-20%,0.007877,0.007877
20-40%,0.000911,0.000911
40-60%,0.009168,0.009168
60-80%,0.012081,0.012081
80+,0.006773,0.007878


,mean,median
ownershipPercBin,,
0-20%,0.024305,0.024305
20-40%,0.026870,0.026870
40-60%,0.038210,0.038210
60-80%,0.026445,0.026445
80+,0.023986,0.024311


,mean,median
ownershipPercBin,,
0-20%,0.053791,0.053791
20-40%,0.058803,0.058803
40-60%,0.082708,0.082708
60-80%,0.050818,0.050818
80+,0.052799,0.053799


,cusip,tic,ownershipPerc,startDate,30,90,180,ownershipPercBin
7,021344475,NBG,0.435601,2000-02-23,0.009168,0.038210,0.082708,40-60%
6,020111623,IGO,0.243968,2000-03-05,0.000911,0.026873,0.058803,20-40%
9,031416767,WWJ,0.389065,2000-03-05,0.000911,0.026868,0.058802,20-40%
8,024571673,RIE,0.869648,2000-03-19,0.007881,0.024308,0.053805,80+
2,007882842,GAC,0.849419,2000-03-19,0.007877,0.024313,0.053804,80+
4,013288385,TPN,0.889731,2000-03-19,0.007880,0.024311,0.053799,80+
1,006016833,IIK,0.070316,2000-03-19,0.007878,0.024309,0.053796,0-20%
10,040762128,VJL,0.920877,2000-03-19,0.007878,0.024311,0.053793,80+
0,003310264,BVX,0.173435,2000-03-19,0.007875,0.024300,0.053786,0-20%
3,012631104,TWE,0.710629,2000-04-09,0.012081,0.026445,0.050818,60-80%


In [ ]:
# Print number of vals in each bin
print(allTradesDfDroppedNan.groupby('ownershipPercBin')['30'].count())
print(allTradesDfDroppedNan.groupby('ownershipPercBin')['90'].count())
print(allTradesDfDroppedNan.groupby('ownershipPercBin')['180'].count())


ownershipPercBin
0-20%     42
20-40%    43
40-60%    42
60-80%    33
80+       40
Name: 30, dtype: int64
ownershipPercBin
0-20%     42
20-40%    43
40-60%    42
60-80%    33
80+       40
Name: 90, dtype: int64
ownershipPercBin
0-20%     42
20-40%    43
40-60%    42
60-80%    33
80+       40
Name: 180, dtype: int64
